## 1)

In [1]:
from ortools.linear_solver import pywraplp as glp   

In [2]:
def optimize(labor_hours):
    '''
    No, I could not think of a better way to do this
    '''    
    mymodel = glp.Solver('BPI', glp.Solver.GLOP_LINEAR_PROGRAMMING)
    
    # define decision variables
    P1 = mymodel.NumVar(0, mymodel.infinity(), 'Product 1')
    P2 = mymodel.NumVar(0, mymodel.infinity(), 'Product 2')
    P3 = mymodel.NumVar(0, mymodel.infinity(), 'Product 3')

    # define objective function
    Profit = mymodel.Objective()
    Profit.SetCoefficient(P1, 30)
    Profit.SetCoefficient(P2, 50)
    Profit.SetCoefficient(P3, 20)

    Profit.SetMaximization()
    M1_hours_constraint = mymodel.Constraint(0, 40)
    M1_hours_constraint.SetCoefficient(P1, .5)
    M1_hours_constraint.SetCoefficient(P2, 2)
    M1_hours_constraint.SetCoefficient(P3, .75)

    M2_hours_constraint = mymodel.Constraint(0, 40)
    M2_hours_constraint.SetCoefficient(P1, 1)
    M2_hours_constraint.SetCoefficient(P2, 1)
    M2_hours_constraint.SetCoefficient(P3, .5)

    # at most 50%
    P1_pct_constraint = mymodel.Constraint(0, mymodel.infinity())
    P1_pct_constraint.SetCoefficient(P1, -0.5) 
    P1_pct_constraint.SetCoefficient(P2, 0.5)
    P1_pct_constraint.SetCoefficient(P3, 0.5)

    # # at least 20%
    P3_pct_constraint = mymodel.Constraint(0, mymodel.infinity())
    P3_pct_constraint.SetCoefficient(P1, -0.2)
    P3_pct_constraint.SetCoefficient(P2, -0.2)
    P3_pct_constraint.SetCoefficient(P3, 0.8)

    Tot_hours_constraint = mymodel.Constraint(0, labor_hours)
    Tot_hours_constraint.SetCoefficient(P1, 2*.5 + 1)
    Tot_hours_constraint.SetCoefficient(P2, 2*2 + 1)
    Tot_hours_constraint.SetCoefficient(P3, 2*.75 + 0.5)
    return mymodel, (P1, P2, P3), Profit, Tot_hours_constraint

### a)

In [3]:
# Solve the model and print optimal solution
mymodel, (P1, P2, P3), Profit, Hours_constraint = optimize(labor_hours=100)
mymodel.Solve()

print('Optimal Solution:')
print('Total Profit = %.2f' % Profit.Value())

print('P1 Quantity = %.2f' % P1.solution_value())
print('P2 Quantity = %.2f' % P2.solution_value())
print('P3 Quantity = %.2f' % P3.solution_value())

# marginal value of the resources
print('Hours constraint = %.2f' % Hours_constraint.dual_value())


Optimal Solution:
Total Profit = 1250.00
P1 Quantity = 25.00
P2 Quantity = 0.00
P3 Quantity = 25.00
Hours constraint = 12.50


### b)

In [4]:
def hours(p1, p2, p3):
    return p1*(2*.5 + 1), p2*(2*2 + 1), p3*(2*.75 + 0.5)

In [5]:
print('Production time on machine 1- {0}, 2- {1}, 3- {2}'.format(*hours(25, 0, 25)))

Production time on machine 1- 50.0, 2- 0, 3- 50.0


### c) 
As per the hours constraint marginal value, the value of an additional hour of labor is 12.5$

### d)

In [6]:
# Solve the model and print optimal solution
mymodel, (P1, P2, P3), Profit, Hours_constraint = optimize(labor_hours=120)
mymodel.Solve()

print('Optimal Solution:')
print('Total Profit = %.2f' % Profit.Value())

print('P1 Quantity = %.2f' % P1.solution_value())
print('P2 Quantity = %.2f' % P2.solution_value())
print('P3 Quantity = %.2f' % P3.solution_value())

# marginal value of the resources
print('Hours constraint = %.2f' % Hours_constraint.dual_value())

Optimal Solution:
Total Profit = 1440.00
P1 Quantity = 24.00
P2 Quantity = 8.00
P3 Quantity = 16.00
Hours constraint = 8.00


# 2)

In [7]:
mymodel = glp.Solver('FC', glp.Solver.GLOP_LINEAR_PROGRAMMING)

# define decision variables
BoughtFrame = mymodel.NumVar(0, mymodel.infinity(), 'Purchased Frames')
BoughtSupport = mymodel.NumVar(0, mymodel.infinity(), 'Purchased Supports')
BoughtStrap = mymodel.NumVar(0, mymodel.infinity(), 'Purchased Straps')
MadeFrame = mymodel.NumVar(0, mymodel.infinity(), 'Produced Frames')
MadeSupport = mymodel.NumVar(0, mymodel.infinity(), 'Produced Supports')
MadeStrap = mymodel.NumVar(0, mymodel.infinity(), 'Produced Straps')

# define objective function
Cost = mymodel.Objective()
Cost.SetCoefficient(BoughtFrame, 51)
Cost.SetCoefficient(BoughtSupport, 15)
Cost.SetCoefficient(BoughtStrap, 7.5)
Cost.SetCoefficient(MadeFrame, 38)
Cost.SetCoefficient(MadeSupport, 11.5)
Cost.SetCoefficient(MadeStrap, 6.5)

Cost.SetMinimization()


Cutting = mymodel.Constraint(0, 60*350)
Cutting.SetCoefficient(MadeFrame, 3.5)
Cutting.SetCoefficient(MadeSupport, 1.3)
Cutting.SetCoefficient(MadeStrap, .8)

Milling = mymodel.Constraint(0, 60*420)
Milling.SetCoefficient(MadeFrame, 2.2)
Milling.SetCoefficient(MadeSupport, 1.7)

Shaping = mymodel.Constraint(0, 60*350)
Shaping.SetCoefficient(MadeFrame, 3.1)
Shaping.SetCoefficient(MadeSupport, 2.6)
Shaping.SetCoefficient(MadeStrap, 1.7)


SupQuota = mymodel.Constraint(5000, mymodel.infinity())
SupQuota.SetCoefficient(BoughtSupport, 1)
SupQuota.SetCoefficient(MadeSupport, 1)

FrameQuota = mymodel.Constraint(5000, mymodel.infinity())
FrameQuota.SetCoefficient(BoughtFrame, 1)
FrameQuota.SetCoefficient(MadeFrame, 1)

StrapQuota = mymodel.Constraint(10000, mymodel.infinity())
StrapQuota.SetCoefficient(BoughtStrap, 1)
StrapQuota.SetCoefficient(MadeStrap, 1)


### a)

In [8]:
mymodel.Solve()
print('Optimal Solution:')
print('Total Cost = %.2f' % Cost.Value(), '\n')

print('BoughtFrame Quantity = %.2f' % BoughtFrame.solution_value()) 
print('BoughtSupport Quantity = %.2f' % BoughtSupport.solution_value()) 
print('BoughtStrap Quantity = %.2f' % BoughtStrap.solution_value()) 
print('MadeFrame Quantity = %.2f' % MadeFrame.solution_value()) 
print('MadeSupport Quantity = %.2f' % MadeSupport.solution_value()) 
print('MadeStrap Quantity = %.2f' % MadeStrap.solution_value(), '\n') 

print('Cutting constraint = %.2f' % Cutting.dual_value())
print('Shaping constraint = %.2f' % Shaping.dual_value())
print('Milling constraint = %.2f' % Milling.dual_value())


Optimal Solution:
Total Cost = 332596.15 

BoughtFrame Quantity = 0.00
BoughtSupport Quantity = 2884.62
BoughtStrap Quantity = 10000.00
MadeFrame Quantity = 5000.00
MadeSupport Quantity = 2115.38
MadeStrap Quantity = 0.00 

Cutting constraint = 0.00
Shaping constraint = -1.35
Milling constraint = 0.00


### b)
The value of another hour in the shaping department is 60*1.35$, and this department is the only bottleneck - no value in more hours in other departments 

In [9]:
mymodel = glp.Solver('FC', glp.Solver.GLOP_LINEAR_PROGRAMMING)

# define decision variables
BoughtFrame = mymodel.NumVar(0, mymodel.infinity(), 'Purchased Frames')
BoughtSupport = mymodel.NumVar(0, mymodel.infinity(), 'Purchased Supports')
BoughtStrap = mymodel.NumVar(0, mymodel.infinity(), 'Purchased Straps')
MadeFrame = mymodel.NumVar(0, mymodel.infinity(), 'Produced Frames')
MadeSupport = mymodel.NumVar(0, mymodel.infinity(), 'Produced Supports')
MadeStrap = mymodel.NumVar(0, mymodel.infinity(), 'Produced Straps')

# define objective function
Cost = mymodel.Objective()
Cost.SetCoefficient(BoughtFrame, 45)
Cost.SetCoefficient(BoughtSupport, 15)
Cost.SetCoefficient(BoughtStrap, 7.5)
Cost.SetCoefficient(MadeFrame, 38)
Cost.SetCoefficient(MadeSupport, 11.5)
Cost.SetCoefficient(MadeStrap, 6.5)

Cost.SetMinimization()


Cutting = mymodel.Constraint(0, 60*350)
Cutting.SetCoefficient(MadeFrame, 3.5)
Cutting.SetCoefficient(MadeSupport, 1.3)
Cutting.SetCoefficient(MadeStrap, .8)

Milling = mymodel.Constraint(0, 60*420)
Milling.SetCoefficient(MadeFrame, 2.2)
Milling.SetCoefficient(MadeSupport, 1.7)

Shaping = mymodel.Constraint(0, 60*350)
Shaping.SetCoefficient(MadeFrame, 3.1)
Shaping.SetCoefficient(MadeSupport, 2.6)
Shaping.SetCoefficient(MadeStrap, 1.7)


SupQuota = mymodel.Constraint(5000, mymodel.infinity())
SupQuota.SetCoefficient(BoughtSupport, 1)
SupQuota.SetCoefficient(MadeSupport, 1)

FrameQuota = mymodel.Constraint(5000, mymodel.infinity())
FrameQuota.SetCoefficient(BoughtFrame, 1)
FrameQuota.SetCoefficient(MadeFrame, 1)

StrapQuota = mymodel.Constraint(10000, mymodel.infinity())
StrapQuota.SetCoefficient(BoughtStrap, 1)
StrapQuota.SetCoefficient(MadeStrap, 1)


In [10]:
mymodel.Solve()
print('Optimal Solution:')
print('Total Profit = %.2f' % Cost.Value(), '\n')

print('BoughtFrame Quantity = %.2f' % BoughtFrame.solution_value()) 
print('BoughtSupport Quantity = %.2f' % BoughtSupport.solution_value()) 
print('BoughtStrap Quantity = %.2f' % BoughtStrap.solution_value()) 
print('MadeFrame Quantity = %.2f' % MadeFrame.solution_value()) 
print('MadeSupport Quantity = %.2f' % MadeSupport.solution_value()) 
print('MadeStrap Quantity = %.2f' % MadeStrap.solution_value(), '\n') 

print('Cutting constraint = %.2f' % Cutting.dual_value())
print('Shaping constraint = %.2f' % Shaping.dual_value())
print('Milling constraint = %.2f' % Milling.dual_value())


Optimal Solution:
Total Profit = 332596.15 

BoughtFrame Quantity = 0.00
BoughtSupport Quantity = 2884.62
BoughtStrap Quantity = 10000.00
MadeFrame Quantity = 5000.00
MadeSupport Quantity = 2115.38
MadeStrap Quantity = 0.00 

Cutting constraint = 0.00
Shaping constraint = -1.35
Milling constraint = 0.00


### c)
even though there is a reduced cost to purchase, there are still zero frames purchased in this case, so it is still not worth buying.